### <div align="center">Федеральное государственное бюджетное образовательное учреждение высшего образования</div>   
### <div align="center"> «ФИНАНСОВЫЙ УНИВЕРСИТЕТ ПРИ ПРАВИТЕЛЬСТВЕ РОССИЙСКОЙ ФЕДЕРАЦИИ»</div>
#### <div align="center">Департамент анализа данных и машинного обучения</div>
<br>
<div align="center">Пояснительная записка к курсовой работе
по дисциплине «Технологии анализа данных и машинное обучение»
на тему:</div>      
<br>
<div align="center">«Разработка системы подбора комплекта мебели на основании запроса на естественном языке»</div>
<br>
<br>
<div align="right">Выполнил:

студент группы ПМ 22-4

факультета информационных технологий и

анализа больших данных

Зевакин И. Е.

Нацчный руководитель:

д.э.н., доцент, профессор</div>

<h3>Бизнес задача</h3>

Я выбрал мебельный сайт магазина "Шатура". Там были взяты шкафы, стулья.

Цель проекта - подборка комплекта мебели, а также для получения информации о самой мебели.

<h2>Этапы проета</h2>

1.   Написание парсера для сайта
  - Собрать нужные данные
2.   Предобработка данных
  - Очистка данных
  - Выбор определённых слов
  - Проверка на одинаковые названия разных товаров
  - Выявление пустых данных
3. Подбор наилучшей мебели для клиента
  - Векторизация данных и запроса
  - Нахождение ближайшего вектора в данных к вектору запроса
4. Выводы и рекомендации



<h3>Выполнение проекта</h3>

Импортируем библиотеки:

In [58]:
!pip install pymorphy2

In [59]:
import pymorphy2
import json
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from multiprocessing import cpu_count
import nltk
import requests as req
from bs4 import BeautifulSoup as BS
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Для получения данных с сайта была испольозвана библиотека bs4.

In [60]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.51 Safari/537.36'
}


class GetHtml:
    def __get__(self, resp, _):
        return BS(resp.text, 'lxml')


req.Response.html = GetHtml()
ses = req.Session()
ses.headers = headers

In [61]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

Далее написаны функции для парсинга.

Первая функция принимает в себя кусок ссылки из третьей функции на определённый предмет и возвращает полную ссылку на него.

In [62]:
def parse_category(link):
    link = 'https://www.shatura.com' + link
    return link

Данная функция принимает в себя полную ссылку на предмет и название предмета. Вохвращает она словарь. В случае ошибки, когда html код сайта отличается от общего, выводится название предмета, который не получилось спарсить.

В данной функции я собираю ссылку на картинку, ссылку на сам предмет, его описание, описание, которое потом пойдёт на векторизацию, цену, размеры, характеристики.

In [63]:
def parse_good(link, good_title):
    try:
        html = ses.get(link, verify=False).html
        img = html.select('.product__swiper-slider .swiper-slide')[0].get('href')
        img = 'https://www.shatura.com' + img
        raw_info = "\n".join([k.strip() for k in html.select('.product-descr__cnt')[0].getText().strip().split('\n')])
        info = word_tokenize(html.select('.product-descr__cnt')[0].getText())
        list_of_words = []
        for word in range(len(info)):
            if morph.parse(info[word])[0].tag.POS not in ['NPRO', 'PREP', 'ADJS', 'ADVB', 'CONJ', 'PRTS', None, 'VERB',
                                                          'INFN', 'PRCL', 'PRED']:
                list_of_words.append(morph.parse(info[word])[0].normal_form)
        info = ' '.join(list_of_words) # сохраняем начальную форму некоторых частей речи для дальнейшей векторизации
        price = html.select('.sidebar__cost .sidebar__price')[0].getText().strip()
        characteristics_st = [chars for chars in html.select('.chars')[0].getText().split('\n') if chars != '']
        characteristics = []
        size = {'Ширина': '', 'Длина': '', 'Высота': ''}
        for char in range(len(characteristics_st)):
            if characteristics_st[char] not in ['Коллекция', 'Стиль', 'Глубина (Ширина)', 'Длина', 'Высота',
                                                'Производитель', 'Производственный код', 'Цвет', 'Оттенок', 'Покрытие',
                                                'Тип полок', 'Тип фасада']:
                characteristics.append(characteristics_st[char])
            if characteristics_st[char] == 'Глубина (Ширина)':
                size['Ширина'] = characteristics_st[char + 1]
            elif characteristics_st[char] == 'Длина':
                size['Длина'] = characteristics_st[char + 1]
            elif characteristics_st[char] == 'Высота':
                size['Высота'] = characteristics_st[char + 1]
        info = good_title.capitalize() + ' ' + info + ' ' + ' '.join([' '.join([item, size[item]]) for item in size])
        return {"Описание": info, "Сырое описание": raw_info, "Цена": price, "Характеристика": characteristics,
                "Размеры": size, "Ссылка на картинку": img, "Ссылка на мебель": link}
    except:
        print(f"Предмет {good_title} не получилось добавить в виду уникального html кода")

Последняя функция парсинга приводит в действие оставльные функции парсинга. на вход получает страницу с каталога, который надо спасить, и все предметы, которые надо будет спасить.

Данная функция записывает всю информацию о предмете в указанный файл. Таким образом получается датасет, который пойдёт на обработку.

In [64]:
def return_info(link, info_list):
    for info in range(len(info_list)):
        info_link = parse_category(link.html.select('.card__title')[info].get('href'))
        product_title = link.html.select('.card__title')[info].getText().lower()
        if 'шкаф' in product_title.split()[0]:
            information = parse_good(info_link, product_title)
            '''file = json.load(open('shkaf.json'))
            if product_title in file:
                if information not in file[product_title]:
                    file[product_title].append(information)
            else:
                file.update({product_title: [information]})
            json.dump(file, open('shkaf.json', 'w'))'''

Данный код написан, чтобы проходиться по нескольким страницам каталога сразу.

In [65]:
for i in range(10, 10+1):
    res = ses.get(f'https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/?PAGEN_1={i}', verify=False).html
    list_of_all_types = res.html.select('.card__title')

    return_info(res, list_of_all_types)

Здесь указана мебель, которая спаршена. Она понадобится для отбора данных для векторизации.

Также здесь указан пользотельский запрос.

In [135]:
known_furniture = ['шкаф', 'гардероб',
                   'стеллаж', 'гардеробная', 'шкаф-стеллаж', 'шкаф-витрина']

morph = pymorphy2.MorphAnalyzer(lang='ru')

user_request = ['Я хочу шкаф-витрину с двумя полками']
request_furniture = []

Функция для обработки запроса пользователя для векторизации.

In [129]:
def format_request(request):
    new_request = []
    for k in request:
        for word in k.split():
            if morph.parse(word)[0].tag.POS not in ['NPRO', 'PREP', 'ADJS', 'ADVB', 'CONJ', 'PRTS', None, 'VERB',
                                                    'INFN', 'PRCL', 'PRED']:
                new_request.append(morph.parse(word)[0].normal_form)
                if morph.parse(word)[0].normal_form in known_furniture:
                    request_furniture.append(morph.parse(word)[0].normal_form)
    return new_request

formated_request = format_request(user_request)

In [130]:
if  'шкаф' in request_furniture[0]:
  with open('shkaf.json', 'r') as f:
      data = f.read()
  new_data = json.loads(data)
elif 'гардероб' in request_furniture[0]:
  with open('garderob.json', 'r') as f:
      data = f.read()
  new_data = json.loads(data)
elif 'стеллаж' in request_furniture[0]:
  with open('stellazh.json', 'r') as f:
      data = f.read()
  new_data = json.loads(data)

Нахождение ключевых слов для исключения лишних данных.

In [131]:
new_list = []
new_dict = []
names = []
for i in new_data:
    for j in range(len(new_data[i])):
        if new_data[i][j] is not None:
            if request_furniture[0] in i and len(list(set(formated_request) & set(i.split()))) > 1:
                new_list.append(new_data[i][j]['Описание'])
                new_dict.append(new_data[i][j])
                names.append(i)

if not new_list:
    for i in new_data:
        for j in range(len(new_data[i])):
            if new_data[i][j] is not None:
                if request_furniture[0] in i:
                    new_list.append(new_data[i][j]['Описание'])
                    new_dict.append(new_data[i][j])
                    names.append(i)

In [132]:
# Токенизация предложений и подготовка данных для обучения
tagged_data = [TaggedDocument(words=word_tokenize(sentence.lower()), tags=[str(i)]) for i, sentence in
               enumerate(new_list)]

# Обучение модели Doc2Vec
model = Doc2Vec(vector_size=5, min_count=2, epochs=200, window=5, workers=cpu_count())
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [133]:
# Получение вектора для нового предложения
new_sentence = format_request(user_request)[0]
new_sentence_vector = model.infer_vector(word_tokenize(new_sentence.lower()))

similar_sentence = model.dv.most_similar([new_sentence_vector], topn=10)
similar_sentence

[('4', 0.8276271820068359),
 ('5', 0.8272932171821594),
 ('2', 0.818568229675293),
 ('1', 0.815552294254303),
 ('11', 0.8104829788208008),
 ('17', 0.8063879013061523),
 ('20', 0.8039999008178711),
 ('0', 0.802068829536438),
 ('13', 0.8010880947113037),
 ('10', 0.7981353402137756)]

Вывод полученных данных.

In [134]:
data_to_db = {}
final_names = []
descriptions = []
price = []
characteristics = []
size = []
img = []
link = []
for i in similar_sentence:
    string = ''
    final_names.append(names[int(i[0])].capitalize())
    descriptions.append(new_dict[int(i[0])]["Сырое описание"])
    price.append(new_dict[int(i[0])]["Цена"])
    characteristics.append('\n'.join(new_dict[int(i[0])]["Характеристика"]))
    for siz in range(3):
      string +=' '.join([k for k in new_dict[int(i[0])]["Размеры"].items()][siz])
    size.append(string)
    img.append(new_dict[int(i[0])]["Ссылка на картинку"])
    link.append(new_dict[int(i[0])]["Ссылка на мебель"])
data_to_db['Название'] = final_names
data_to_db['Описание'] = descriptions
data_to_db["Цена"] = price
data_to_db['Характеристики'] = characteristics
data_to_db['Размеры'] = size
data_to_db['Ссылка на картинку'] = img
data_to_db['Ссылка на мебель'] = link
db = pd.DataFrame(data_to_db)
db[['Название', 'Ссылка на мебель']]

,Название,Ссылка на мебель
0,"Шкаф-витрина одностворчатый chelsea,rimini",https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/488166/
1,"Шкаф-витрина одностворчатый chelsea,rimini",https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/488168/
2,"Шкаф-витрина одностворчатый chelsea,rimini",https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/488165/
3,"Шкаф-витрина одностворчатый chelsea,rimini",https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/488175/
4,Шкаф-витрина одностворчатый soho беж,https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/487062/
5,Шкаф-витрина одностворчатый soho белая,https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/487855/
6,Шкаф-витрина одностворчатый soho белая,https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/487858/
7,"Шкаф-витрина одностворчатый chelsea,rimini",https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/488176/
8,Шкаф-витрина одностворчатый soho беж,https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/487058/
9,Шкаф-витрина одностворчатый soho беж,https://www.shatura.com/goods/groups/shkafy_stellazhi_polki/487061/


In [136]:
while True:
  print('1 - Название')
  print('2 - Описание')
  print('3 - Цена')
  print('4 - Характеристики')
  print('5 - Размеры')
  print('6 - Ссылка на картинку')
  print('7 - Ссылка на мебель')
  print('8 - нет')
  try:
    user_input = input('Хотите что-нибудь узнать? (Введите цифру опции)')
    print()
    if user_input == '8':
      break
    user_input_2 = int(input('О каком прдмете хотите узнать больше? (Введите цифру прдмета в выведенном датасете)'))
    print()
    print()
    if user_input == '1':
      print(db['Название'][user_input_2])
      print()
      print()
    elif user_input == '2':
      print(db['Описание'][user_input_2])
      print()
      print()
    elif user_input == '3':
      print(db['Цена'][user_input_2])
      print()
      print()
    elif user_input == '4':
      print(db['Характеристики'][user_input_2])
      print()
      print()
    elif user_input == '5':
      print(db['Размеры'][user_input_2])
      print()
      print()
    elif user_input == '6':
      print(db['Ссылка на картинку'][user_input_2])
      print()
      print()
    elif user_input == '7':
      print(db['Ссылка на мебель'][user_input_2])
      print()
      print()
  except:
    print('Вы пытались как-то сломать меня. Пожалуйста, больше так не делаете.')
    print()
    print()

1 - Название
2 - Описание
3 - Цена
4 - Характеристики
5 - Размеры
6 - Ссылка на картинку
7 - Ссылка на мебель
8 - нет
Хотите что-нибудь узнать? (Введите цифру опции)1

О каком прдмете хотите узнать больше? (Введите цифру прдмета в выведенном датасете)0


Шкаф-витрина одностворчатый chelsea,rimini


1 - Название
2 - Описание
3 - Цена
4 - Характеристики
5 - Размеры
6 - Ссылка на картинку
7 - Ссылка на мебель
8 - нет
Хотите что-нибудь узнать? (Введите цифру опции)2

О каком прдмете хотите узнать больше? (Введите цифру прдмета в выведенном датасете)1


Наша мебель производится только из экологичных, надежных и качественных материалов преимущественно ведущих европейских производителей. Мебель сертифицирована. Вся фурнитура для сборки и крепления в комплекте. Срок службы мебели - 20лет, гарантийный срок эксплуатации - до 5 лет.

Описание: Шкаф-витрина подходит для хранения красивой праздничной посуды, любимых и памятных вещей, различных редких сувениров, наград, а также для зонирования комна

<h2>Выводы</h2>

Передо мной стояла задача вывести комплект для мебели. Он находится в описании.

В ходе выполнения курсовой, я изучил множетсво сайтов, как их парсить. Также я рассмотрел библиотеку Doc2Vec для векторизации данных и нахождения ближайших к векторизированному запросу.

Было построено несколько моделей, одна из которых была на полном датасете, но от неё я отказался в связи с тем, что на естественном запросе человека возмжожен непредвиденный вывод данных. В связи с этим данные были рахделены на несколько датасетов.

В выводе similar_sentence можно увидеть насколько векторизированный запрос подходил под векторизированные данные прдмета.0


<h2>Рекомендации</h2>

Анализ показал, что html код сайта "Шатура" не является наилучшим: в каких-то местах нет одного патерна html кода, нет определённого патерна описания.

Изначально хотелось рассмотреть сайт "Hoff" или "Яндекс маркет", но по причине того, что сайты имели какую-то блокировку к парсингу, спарсить их не получилось.

Также в отличие от "Hoff" и "Яндекс маркета" на "Шатуре" не было определённости в компонентах: где-то были указаны все детали, где-то было указано количество компонентов, но не всех, где-то компоненты были указаны в описании.

Хочется сказать ещё, что очень важно, как пользователь будет вводить запрос. Чем меньше запрос пользователя, тем больше разброс по датасету. Поэтому рекомендуется указывать как можно быльше информации о мебели, которую хочется приобрести.